<a href="https://colab.research.google.com/github/ParitoshP702/Bilevel-Optimization/blob/main/CIFAR_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Sun Jan  8 15:29:27 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   29C    P0    53W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
(X_train,Y_train),(X_test,Y_test) = tf.keras.datasets.cifar10.load_data()

170498071/170498071 [==============================] - 2s 0us/step


In [ ]:
# pip install numba

In [ ]:
# from numba import cuda
# device = cuda.get_current_device()
# device.reset()

In [ ]:
train_count = 2000
eval_count = 1000
test_count = 1000

In [ ]:
X_train = X_train/255.0
X_test  = X_test/255.0

In [ ]:
# x_train[0]

In [ ]:
x_train = X_train[:train_count,:,:,:]
x_eval = X_train[train_count:train_count+eval_count,:,:,:]
y_train = Y_train[:train_count]
y_eval = Y_train[train_count:train_count+eval_count]

In [ ]:
x_test = X_test[:test_count,:,:,:]
y_test = Y_test[:test_count]

In [ ]:
#Flattening our datasets
x_train = x_train.reshape(x_train.shape[0],-1)
x_eval = x_eval.reshape(x_eval.shape[0],-1)
x_test = x_test.reshape(x_test.shape[0],-1)

In [ ]:
y_training = np.zeros(shape = (len(y_train),10), dtype = float)#one hot encoding the training labels
for i in range(len(y_train)):
  for j in range(10):
    if j  == y_train[i]:
      y_training[i][j] = 1.0
    else:
      y_training[i][j] = 0.0

In [ ]:
y_val_array = np.zeros(shape = (len(y_eval),10),dtype  =float)#one hot encoding the validation labels
for i in range(len(y_eval)):
  for j in range(10):
    if j  == y_eval[i]:
      y_val_array[i][j] = 1.0
    else:
      y_val_array[i][j] = 0.0

In [ ]:
y_val_array

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
# y_eval

In [ ]:
loss_object = tf.keras.losses.CategoricalCrossentropy()

In [ ]:
x_train.shape

(2000, 3072)

In [ ]:
number_of_layers = 5
neurons_per_layer = 50
optimizer = "Adam"
activation_function = "sigmoid"

In [ ]:
y_training

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [ ]:
model = Sequential()
model.add(Dense(units = 2,input_dim = x_train.shape[1]))
for _ in range(number_of_layers):
  model.add(Dense(units = neurons_per_layer,activation = activation_function, kernel_regularizer = tf.keras.regularizers.L2(1e-10)))
model.add(Dense(units = 10, activation = "softmax",kernel_regularizer = tf.keras.regularizers.L2(1e-10)))
model.compile(loss = tf.keras.losses.CategoricalCrossentropy(),optimizer = optimizer)
model.fit(x_train,y_training,epochs = 10,batch_size = 64)

Epoch 1/10
32/32 [==============================] - 4s 3ms/step - loss: 2.3358
Epoch 2/10
32/32 [==============================] - 0s 3ms/step - loss: 2.3050
Epoch 3/10
32/32 [==============================] - 0s 3ms/step - loss: 2.2919
Epoch 4/10
32/32 [==============================] - 0s 3ms/step - loss: 2.2482
Epoch 5/10
32/32 [==============================] - 0s 3ms/step - loss: 2.1795
Epoch 6/10
32/32 [==============================] - 0s 3ms/step - loss: 2.1269
Epoch 7/10
32/32 [==============================] - 0s 3ms/step - loss: 2.0939
Epoch 8/10
32/32 [==============================] - 0s 3ms/step - loss: 2.0740
Epoch 9/10
32/32 [==============================] - 0s 3ms/step - loss: 2.0540
Epoch 10/10
32/32 [==============================] - 0s 3ms/step - loss: 2.0381


In [ ]:
y_pred = model(x_eval,training = False)

In [ ]:
loss_object(y_pred,y_val_array)

<tf.Tensor: shape=(), dtype=float64, numpy=13.776153750404498>

In [ ]:
final_hessian_list = []

In [ ]:
# g

In [ ]:
with tf.GradientTape(persistent = True) as tape1:
  with tf.GradientTape(persistent = True) as tape2:
    loss_object = tf.keras.losses.CategoricalCrossentropy()
    y_pred_array = model(x_train,training = True)
    loss = loss_object(y_training,y_pred_array)
  g = tape2.gradient(loss, model.trainable_variables)
for i in range(len(g)):
  # reshaped_grad = tf.reshape(g[i], [-1])
  h = tape1.jacobian(g[i],model.trainable_variables)
  final_hessian_list.append(h)

In [ ]:
total_weights = 0
for i in range(len(model.weights)):
  if i%2 == 0:
    total_weights += model.weights[i].shape[0]*model.weights[i].shape[1]
  else:
    total_weights += model.weights[i].shape[0]

In [ ]:
total_weights

11906

In [ ]:
#

In [ ]:
def compute_gradient(x_target,y_target,model):##general function which returns the list of gradient vector as an numpy array
  with tf.GradientTape() as tape:
    loss_object = tf.keras.losses.MeanSquaredError()
    y_pred_array = model(x_target,training = True)
    loss = loss_object(y_target,y_pred_array)
  g = tape.gradient(loss,model.trainable_variables)
  final_grad_list = []
  for i in range(len(g)):
    grad_array = tf.make_ndarray(tf.make_tensor_proto(g[i]))
    if i%2==0:
      grad_shape = grad_array.shape
      for j in range(grad_shape[0]):
        for k in range(grad_shape[1]):
          final_grad_list.append(grad_array[j][k])
    else:
      length = grad_array.shape[0]
      for j in range(length):
        final_grad_list.append(grad_array[j])
  return np.array(final_grad_list)


In [ ]:
grad_validation =  compute_gradient(x_eval,y_val_array,model)

In [ ]:
#reshaping the double dimensional list of tensors into a matrix
hessian_matrix = np.empty(shape = (1,1),dtype = float)
for i in range(len(final_hessian_list)):
  hess_col_mat = np.empty(shape = (1,1),dtype = float)
  for j in range(len(final_hessian_list[i])):
    hess_array = tf.make_ndarray(tf.make_tensor_proto(final_hessian_list[i][j]))
    hess_shape = hess_array.shape
    if i%2 == 0:
       if j%2 == 0:
         hess_array = hess_array.reshape(hess_shape[0]*hess_shape[1],hess_shape[2]*hess_shape[3])
       else:
         hess_array = hess_array.reshape(hess_shape[0]*hess_shape[1],hess_shape[2])
    else:
      if j%2 == 0:
        hess_array = hess_array.reshape(hess_shape[0],hess_shape[1]*hess_shape[2])
      else:
        hess_array = hess_array
    if j==0 :
      hess_col_mat = hess_array
    else:
      hess_col_mat = np.concatenate((hess_col_mat,hess_array),axis = 1)
  if i==0:
    hessian_matrix = hess_col_mat
  else:
    hessian_matrix= np.concatenate((hessian_matrix,hess_col_mat),axis = 0)



In [ ]:
hessian_matrix.shape

(11906, 11906)

In [ ]:
grad_validation

array([ 2.5951855e-05, -6.8204417e-06,  2.3988538e-05, ...,
       -1.8154573e-05,  9.8558899e-05,  5.9389917e-05], dtype=float32)

In [ ]:
skip_length = len(model.layers[0].weights[0].numpy().reshape(-1)) + len(model.layers[0].weights[1].numpy().reshape(-1))

In [ ]:
skip_length

6146

In [ ]:
weights_list = []
weights_list_new = []

In [ ]:
for i in range(len(model.weights)):
  weights_array = tf.make_ndarray(tf.make_tensor_proto(model.weights[i]))
  if i%2 == 0:
    shape_array = weights_array.shape
    for j in range(shape_array[0]):
      for k in range(shape_array[1]):
        weights_list.append(weights_array[j][k])
        if len(weights_list_new) < skip_length:
          weights_list_new.append(0)
        else:
          weights_list_new.append(weights_array[j][k])
  else:
    lgt = weights_array.shape[0]
    for j in range(lgt):
      weights_list.append(weights_array[j])
      if len(weights_list_new) < skip_length:
        weights_list_new.append(0)
      else:
        weights_list_new.append(weights_array[j])

In [ ]:
final_weights_array = np.array(weights_list)
final_weights_array_new = np.array(weights_list_new)

In [ ]:
l = len(final_weights_array)

In [ ]:
weight_array_vec = final_weights_array_new.reshape(l,1)/len(y_train)

In [ ]:
hessian_col_mat = np.concatenate((weight_array_vec,hessian_matrix),axis = 1)

In [ ]:
weight_array_withreg = np.concatenate(([[0]],final_weights_array_new.reshape(1,l)),axis = 1)/len(y_train)

In [ ]:
hessian_mat_with_reg = np.concatenate((weight_array_withreg,hessian_col_mat),axis = 0)

In [ ]:
hessian_mat_with_reg

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         4.65580402e-06,  7.91620743e-06, -7.85000250e-06],
       [ 0.00000000e+00,  3.42549733e-03, -1.70161983e-03, ...,
         1.14310347e-03, -5.09141991e-03, -4.02774755e-03],
       [ 0.00000000e+00, -1.70161983e-03,  1.26674864e-03, ...,
        -8.70586664e-04,  2.76647857e-03,  3.02729453e-03],
       ...,
       [ 4.65580402e-06,  1.14308833e-03, -8.70564720e-04, ...,
         8.76334459e-02, -7.25208316e-03, -7.99116865e-03],
       [ 7.91620743e-06, -5.09129511e-03,  2.76647462e-03, ...,
        -7.25208269e-03,  8.06169659e-02, -1.95568372e-02],
       [-7.85000250e-06, -4.02775407e-03,  3.02725215e-03, ...,
        -7.99116865e-03, -1.95568372e-02,  8.59261528e-02]])

In [ ]:
grad_validation_new=[]

In [ ]:
grad_validation_new = np.concatenate(([[0]],grad_validation.reshape(1,l)),axis = 1)

In [ ]:
##obtaining the new gradient array
grad_validation_new = grad_validation_new.reshape(-1)

In [ ]:
ub = [10 for i in range(l+1)]

In [ ]:
lb = []
for i in range(l+1):
  if i==0:
    lb.append(1e-5)
  else:
    lb.append(-10)

In [ ]:
pip install gurobipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 87.8 MB/s eta 0:00:00


In [ ]:
import gurobipy as gp

In [ ]:
from gurobipy import *

In [ ]:
params = {
"WLSACCESSID": '753e7886-7142-449d-8baa-d41ca78716ef',
"WLSSECRET": '880d2525-364b-41d0-ac23-6dcf7ad15312',
"LICENSEID": 914249,
}
env = gp.Env(params=params)

# Create the model within the Gurobi environment
m = gp.Model(env=env)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 914249
Academic license - for non-commercial use only - registered to ppankaj21@iitk.ac.in


In [ ]:
# m = gp.Model()
x = m.addMVar((l+1,),lb = lb, ub = ub )
m.setObjective(grad_validation_new@x)
# m.addConstr(hessian_mat_with_reg@x == 0)
m.addConstr(hessian_mat_with_reg@x <= 0.1)
m.addConstr(hessian_mat_with_reg@x >= -0.1)
x.PStart = np.zeros(l+1)
# GRBModel.Set(Pstart = np.zeros(l+1))
m.optimize()

Warning on update mode = 1 after new variables or constraints added:
Setting LP warm start basis or start ignored
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Academic license - for non-commercial use only - registered to ppankaj21@iitk.ac.in
Optimize a model with 23814 rows, 11907 columns and 283528712 nonzeros
Model fingerprint: 0x4be768fa
Coefficient statistics:
  Matrix range     [3e-13, 1e-01]
  Objective range  [2e-09, 8e-04]
  Bounds range     [1e-05, 1e+01]
  RHS range        [1e-01, 1e-01]
Presolve removed 0 rows and 0 columns (presolve time = 18s) ...
Presolve removed 0 rows and 0 columns (presolve time = 26s) ...
Presolve removed 0 rows and 0 columns (presolve time = 44s) ...
Presolve removed 0 rows and 0 columns (presolve time = 98s) ...
Presolve removed 0 rows and 0 columns (presolve time = 142s) ...

In [ ]:
weight_array_with_reg = np.concatenate(([[1e-10]],final_weights_array.reshape(1,l)),axis = 1).reshape(-1)

In [ ]:
weight_array_with_reg

array([ 1.00000000e-10,  3.48446146e-02,  5.18757887e-02, ...,
        9.31160804e-03,  1.58324149e-02, -1.57000050e-02])

In [ ]:
descent_factors = []
for i in range(-100,30000,200):
  descent_factors.append(i*1e-3)
descent_factors = np.array(descent_factors)

In [ ]:
weight_sample_space_matrix = np.empty(shape = (len(descent_factors),len(weight_array_with_reg)),dtype = float)

In [ ]:
x

<MVar (11907,)>
array([<gurobi.Var C0 (value 10.0)>, <gurobi.Var C1 (value -10.0)>,
       <gurobi.Var C2 (value 10.0)>, ...,
       <gurobi.Var C11904 (value -10.0)>,
       <gurobi.Var C11905 (value 10.0)>,
       <gurobi.Var C11906 (value -10.0)>])

In [ ]:
all_vars = m.getVars()
values = m.getAttr("x",all_vars)

In [ ]:
values = np.array(values)

In [ ]:
values = values/np.linalg.norm(values)

In [ ]:
values

array([ 0.00925867, -0.00925867,  0.00925867, ..., -0.00925867,
        0.00925867, -0.00925867])

In [ ]:
for i in range(len(descent_factors)):
  weight_sample_space_matrix[i] =weight_array_with_reg+ values*descent_factors[i]

In [ ]:
weight_sample_space_matrix

array([[-0.00092587,  0.03577048,  0.05094992, ...,  0.01023747,
         0.01490655, -0.01477414],
       [ 0.00092587,  0.03391875,  0.05280166, ...,  0.00838574,
         0.01675828, -0.01662587],
       [ 0.0027776 ,  0.03206701,  0.05465339, ...,  0.00653401,
         0.01861002, -0.01847761],
       ...,
       [ 0.2731307 , -0.23828608,  0.32500649, ..., -0.26381909,
         0.28896311, -0.2888307 ],
       [ 0.27498243, -0.24013782,  0.32685822, ..., -0.26567082,
         0.29081485, -0.29068244],
       [ 0.27683416, -0.24198955,  0.32870995, ..., -0.26752256,
         0.29266658, -0.29253417]])

In [ ]:
def validation_loss_computation(weight_and_reg_array):##function which computes the loss score of the model corresponding to given weights

  model_demo = Sequential()
  model_demo.add(Dense(units = 2, input_dim = 3072))
  for i in range(number_of_layers):
       model_demo.add(Dense(units = neurons_per_layer, activation = activation_function, kernel_regularizer = tf.keras.regularizers.L2(weight_and_reg_array[0])))
  model_demo.add(Dense(units = 10,activation = "softmax", kernel_regularizer = tf.keras.regularizers.L2(weight_and_reg_array[0])))
  model_demo.compile(loss = "mean_squared_error", optimizer = optimizer, metrics = ["accuracy"])
  weight_tracker = 1##as "weight_and_reg_array" is a one dimensional array it keeps track of the indices of the array
  for i in range(len(model_demo.layers)):##changing the weights of the model layer wise
    orignal_weight_list = model.layers[i].weights
    array_1 = orignal_weight_list[0].numpy()##array corresponding to the weight matrix
    array_2 = orignal_weight_list[1].numpy()##array corresponding to the bias vector
    array_1_new = weight_and_reg_array[weight_tracker:weight_tracker+array_1.shape[0]*array_1.shape[1]]
    weight_tracker += array_1.shape[0]*array_1.shape[1]##updating the weight tracker
    array_2_new = weight_and_reg_array[weight_tracker:weight_tracker + array_2.shape[0]]
    weight_tracker += array_2.shape[0] #updating the weight tracker
    array_1_new = array_1_new.reshape(array_1.shape) ##new weight matrix
    array_2_new = array_2_new.reshape(array_2.shape) ##new bias vector
    list_of_new_array = [] ##list of the new weight matrix and the new bias vector
    list_of_new_array.append(array_1_new)
    list_of_new_array.append(array_2_new)
    model_demo.layers[i].set_weights(list_of_new_array) ##appending the new weights into the given layer of the model
  y_pred_array = model_demo(np.array(x_eval),training = False)
  y_pred_training = model_demo(np.array(x_train),training = False)
  loss = loss_object(y_val_array,y_pred_array)
  loss_t = loss_object(y_training,y_pred_training)
  # loss1,_ = model_demo.evaluate(x_eval,y_eval,verbose= 0)
  # loss2,_= model_demo.evaluate(x_train,y_train,verbose = 0)
  return loss,loss_t,model_demo

In [ ]:
loss_array_valid = np.empty(shape = len(descent_factors),dtype = float)##array to contain the training losses
loss_array_train = np.empty(shape = len(descent_factors),dtype = float)##array to contain the validation losses

for i in range(len(loss_array_valid)):
  loss_array_valid[i] ,loss_array_train[i],_= validation_loss_computation(weight_sample_space_matrix[i])


In [ ]:
y_pred_array = model(x_eval,training = True)

In [ ]:
loss_object(y_val_array,y_pred_array)

<tf.Tensor: shape=(), dtype=float32, numpy=2.0507596>

In [ ]:
ideal_weight_array = weight_sample_space_matrix[loss_array_valid.argmin()]

In [ ]:
_,_,best_model = validation_loss_computation(ideal_weight_array)

In [ ]:
y_pred_testing = best_model(x_test,training = True)

In [ ]:
y_testing = np.zeros(shape = (len(y_test),10))

In [ ]:
loss_array_valid.min()

1.9735487699508667

In [ ]:
for i in range(len(y_test)):
   for j in range(10):
    if j != (y_test[i]):
      y_testing[i][j] = 0.0
    else:
      y_testing[i][j] = 1.0

In [ ]:
loss_object(y_testing,y_pred_testing)

<tf.Tensor: shape=(), dtype=float32, numpy=2.0284164>

In [ ]:
y_pred_test = model(x_test,training = True)

In [ ]:
loss_object(y_testing,y_pred_test)

<tf.Tensor: shape=(), dtype=float32, numpy=2.0609362>

In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame([loss_array_valid,descent_factors,loss_array_train]).transpose().to_csv("CIFAR(temp).csv")